<div style='background-color:#f7f7f7; padding-top:30px; padding-left:20px; padding-right:20px; padding-bottom:30px'>
    <center>
        <div style='  display: block;
  font-size: 2em;
  font-weight: bold;  display: block;
  font-size: 2em;
  font-weight: bold;'>LCP-B - Processing of SEVN data for binary black holes mass distribution analysis
        </div>
    <center>
        <br>
    <i>Tommaso Bertola, Giacomo Di Prima, Giuseppe Viterbo, Marco Zenari</i></center>
    <center>
    <i>All authors contributed equally to the project</i></center>
    

# The physical problem

<div class="square">
    <div>
      <img src=
"Figures/lcp/SEVN.jpg"
        alt="Longtail boat in Thailand" style="float: right; margin-left:20px" width=50%>
    </div>
      
<p style="text-align: justify; padding-right:10px">
In the era of Gravitational Waves detector of the LIGO-Virgo-KAGRA collaboration sensible to $10^2$ Hz, the study of binary compact objects has become of extreme importance, since it can provide a landscape of the parameters space of the progenitor of these phenomena. The subject of our studies are binary black holes obtained by the population synthesis code SEVN <a name="cite_note-1"><sup>[1]</sup></a>, which interpolates stellar tracks to deal with stellar evolution, and it also takes into account supernova explosions and  binary evolution processes. In particular we aim to understand the importance of the input features of the program in guiding the evolution, and we also try to reproduce the final mass distribution, and some quantities correlated to it. Moreover we aim to define a possible region of the parameters space for which our model prediction could be use as proxy of the SEVN output, cutting the time of computation.
    </p>
  
  </div>
  
  <a name="cite_note-1"></a>1. [^](#cite_note-1) Presentation papers:  <a href="https://ui.adsabs.harvard.edu/abs/2023MNRAS.tmp.1606I/abstract">Iorio et al., 2022</a>, and  <a href="https://ui.adsabs.harvard.edu/abs/2019MNRAS.485..889S/abstract">Spera et al., 2019 </a>.

## Black holes and Binary evolution
Before starting with the description of the algorithm that we use, it is important to understand what exatly are black holes, and how stars evolve into them.

A black hole is a region of the spacetime so dense that nothing has energy high enough to escape from it.
A black hole is formed when a Mass $M$ is within its Schwarzschild Radius $r_s$:

<center>
<div style='background-color:#f7f7f7; text-align: center; max-width: 200px; max-height: 50px; padding-top:30px; padding-left:20px; padding-right:20px; padding-bottom:30px; '>
    $$r_s =  \frac{2GM}{c^2} \approx 3 \big(\frac{M}{M_\odot}\big) \text{km}$$ 
</div></center>

When a star ends its nuclear burning material, no sorce of energy can support the gravity pressure and the star goes out of the hydrostatic equilibrium<a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2), and the core collapse begins. Also, if the Mass of the star is bigger than $20 M_{\odot}$ the gravity is so strong that no source of pressure can stop the core collpase, and a stellar mass black hole is formed. 
As we know from observations, almost all the massive stars ($M$ >  $20 M_{\odot}$) are in binaries (or higher multiplicity systems), so it is important to understand the evolution of binary system; more specifically, SEVN runs are done with isolated binaries, which means that the two stars form from the same cloud and evolve in a gravitationally bound system.

<a name="cite_note-2"></a>2. [^](#cite_ref-2) Hydrostatic equilibrium is described as the balance between Pressure and Nuclear forces and Gravity.


## Our main focus

###  What we try to reproduce
From the black hole mass ($M_{bh1}$, $M_{bh2}$ with $M_{bh1}$> $M_{bh2}$), the metallicity $Z$, the semimajor axis $a$, the eccentricity $e$ and the $\alpha$ parameter, described below, we try to infere the distribution of two quantities: 
The Mass ratio $q$ of the black holes and the Chirp mass $\mathcal{M}$. From an observational point the Chirp mass $\mathcal{M}$ can be also obtain from Gravitational Wave frequency $f$, and its time derivative $\dot{f}$. These two quantities are defined as:
<center>
<div style="background-color:#f7f7f7; text-align: center; max-width: 600px; max-height: 50px; padding-top:30px; padding-left:20px; padding-right:20px; padding-bottom:30px; ">
   $$q = \frac{M_{bh2}}{M_{bh1}} \quad \mathcal{M} =  \frac{(M_{bh1} M_{bh2})^{3/5}}{(M_{bh1} + M_{bh2})^{1/5}} = \frac{c^3}{G} \big( \frac{5}{96} \pi^{-8/3} f^{-11/3} \dot{f} \big)^{3/5} $$ 
</div></center>


### How supernovae influence our predictions
The core collapse is one fondamental ingredient to define the final mass of a black hole. During the final stages of evolution the Silicon burning shell adds material to the Iron core, and when this one gets to a total mass bigger than the Chandrasekar Mass ($1.3-1.7 M_{\odot}$), the degenerate electron pressure is not able to sustain anymore the gravity, and so the runaway core collapse begins. 

When the shock wave is not stopped the star explode as a Supernova, leaving a compact remant, either a Neutron star or a low-mass black hole. If the envelope binding energy is bigger than the Supernova energy the shock wave stops and there is no Supernova explosion, and the star collapses to a black hole of higher mass<a name="cite_ref-3"></a>[<sup>[3]</sup>](#cite_note-3); this process is called Direct Collapse. In SEVN the model of the supernova can be described using different formalism and the one used in the data that we analyze is `rapid_gauNS`<a name="cite_ref-4"></a>[<sup>[4]</sup>](#cite_note-4).

When a Supernova occurs, asymmetry in mass ejection or in the neutrino emission, or symmetric mass loss in binary system<a name="cite_ref-5"></a>[<sup>[5]</sup>](#cite_note-5), leads to the so called `Kick`, which is the gained momentum obtained by the compact object after the Supernova. Unfortunately for binary black holes supernova kicks there is no real observation. The kick velocity $v_k$ is extracted from a Maxwellian distribution and, to obtain the `effective` value $v_{ek}$<a name="cite_ref-4"></a>[<sup>[4]</sup>](#cite_note-4), it is corrected by the fraction of matter $f_b$ that falls back on the proto compact object as follows:
<center>
<div style="background-color:#f7f7f7; text-align: center; max-width: 200px; max-height: 50px; padding-top:30px; padding-left:20px; padding-right:20px; padding-bottom:30px; ">
    $$v_{ek} =  v_k(1-f_b)$$ 
</div></center>

It is important to notice that when $f_b = 1$ a Direct Collpase is triggered, no mass is ejected, and then $v_{ek} = 0$, which means that there is no kick.


Since the kick introduces a stochastic process in the evolution of the binary, in our algorithm we try to derive, a posteriori, how much this kind of phenomena  deviates our prediction of the output mass distribution.


<a name="cite_note-3"></a>3. [^](#cite_ref-3) <a href="https://ui.adsabs.harvard.edu/abs/2001ApJ...554..548F/abstract"> Fryer C. and Kalogera V., 2001</a></br>
<a name="cite_note-4"></a>4. [^](#cite_ref-4) Formalism described in<a href="https://iopscience.iop.org/article/10.1088/0004-637X/749/1/91"> Fryer C. and et al., 2012</a>, but the mass of neutron star are drawn from a Gaussian
</br><a name="cite_note-5"></a>5. [^](#cite_ref-5) <a href="https://ui.adsabs.harvard.edu/abs/1961BAN....15..265B/abstract"> Blaauw, A. , 1961</a>


### How evolution is perturbed in a binary system 
Binary stars can exchange mass in a varity of channels. We investigate the number of Roche Lobe Overflow (RLO) and the number of Common Envelope (CE) that the data undergoes. These two kind of events are briefly described below.

#### Roche Lobe Overflow (RLO)
Assuming the binary orbit is circular, if we want to study the binary system using a non-inertial frame of reference that co-rotate with the binary (so that the stars position is fixed in time) we need to define the Roche Potential $\phi_R$: 

<center>
<div style='background-color:#f7f7f7; text-align: center; max-width: 350px; max-height: 50px; padding-top:30px; padding-left:20px; padding-right:20px; padding-bottom:30px; '>
    $$\phi_{R} =  \frac{G M_1}{|\vec{r} - \vec{d_1}|} + \frac{G M_2}{|\vec{r} - \vec{d_2}|} + \frac{1}{2} |\vec{\omega} \times \vec{r} |^2 $$ 
</div></center>

The Roche lobes are defined as the two lobes of the 8-shape critical potential surface, connected in the middle by the inner Lagrangian point L1. It is defined as `Roche Lobe Overflow` the flow of matter without energy change from the donor to the accretor, and it happens when the donor starts filling its Roche lobe (its radius is equal or larger than the Roche lobe radius). In SEVN the RLO stability is determined by the mass ratio of the two stars and by a parameter that depends on the evolution phase of the donor.
If the RLO is unstable there are two possible outcomes:
* if the stars do no have a core, they merge 
* if one of the two has a core, the stars enter in a `Common Envelope` phase


#### Common Envelope (CE)
As described before, when the RLO becomes unstable the envelope of the donor is shared between the remaing cores. Dragged by the envelope, the two cores start to spiral inwards (efficient way of shrinking a binary). This process can lead to the merging of the two core into one single star, or, if the energy released during the spiral removes the envelope, the two cores form a new tighter binary. In order to describe this phenomena, the $\alpha$ formalism<a name="cite_ref-6"></a>[<sup>[6]</sup>](#cite_note-6) is use by SEVN. The `alpha` parameter is the energy removal efficiency of the Common Envelope and is a free parameter that regulates variation of orbital energy of the cores, before and after the common envelope, $\Delta E_{orb} = \alpha (E_{orb, f} - E_{orb, i})$, where $E_{orb}$ is defined as:

<center>
<div style='background-color:#f7f7f7; text-align: center; max-width: 300px; max-height: 50px; padding-top:30px; padding-left:20px; padding-right:20px; padding-bottom:30px; '>
    $$E_{orb, k} =\frac{1}{2} \frac{G M_{c,1} M_{c,2}}{a_k} \quad  k \in \{ i, f\}$$
</div></center> 

$M_{c,i}$ are the mass of the cores and $a_k$ is the semimajor axis of the cores binary.

<a name="cite_note-6"></a>6. [^](#cite_ref-6) <a href="https://ui.adsabs.harvard.edu/abs/2002MNRAS.329..897H/abstract"> Hurley et al., 2002</a>


## Parameters used for the algorithm
For this reason we examin the following parameter space:

|Parameter type| Tested values| 
|:----|:----|
| [$M_0, M_1$]  | Initial masses of the stars in the binary |
| $Z$         | Metallicity of the two starts in the binary |
| $a$         | Semimajor axis of the binary               |
| $e$         | Eccentricity of the binary                  |
| $\alpha$  | Energy removal efficency of the CE            |

# A brief recap on the preprocessing of the data
Given the initial 1.7TB of data, we need to first preprocess it in order to extract only the meaningful information.
To achieve this goal we use a cluster of 6 VMs hosted in CloudVeneto leveraging the capabilities of Python Dask library.

We show below the implementation of the code used to parse the whole dataset. The data extracted from this algorithm is then saved onto disk to avoid rerunning the whole computation which takes about 2 hours to complete.

In [ ]:
# output_{}.csv
output_column_to_read = ['name', 'Mass_0', 'RemnantType_0',
                         'Mass_1', 'RemnantType_1',
                         'Semimajor','Eccentricity',
                         'GWtime','BWorldtime']
output_column_type = ['string', 'float64', 'int64',
                      'float64', 'int64',
                      'float64', 'float64',
                      'float64', 'float64']

# evolved_{}.dat
evolved_column_to_read = ['name', 'Mass_0',
                          'Z_0', 'SN_0',
                          'Mass_1', 'SN_1',
                          'a', 'e']
evolved_column_type = ['string', 'float64',
                       'float64', 'string',
                       'float64', 'string',
                       'float64', 'float64']

# further columns to remove at the end 
drop_list = ['RemnantType_0',  'RemnantType_1']

In [ ]:
#OUTPUT files processing
output = pd.read_csv(paths[0],                              # read the file
                     usecols=output_column_to_read,         # read only some cols
                     dtype=dict(zip(output_column_to_read,  # specify the types
                                    output_column_type))).\ #
            rename(columns={'Mass_0':'Mass_0_out',          # rename columns
                            'Mass_1':'Mass_1_out'})         #

# mask to select only the black holes binaries, defined by RemnantType
idxBHBH=(output.RemnantType_0==6) &\
        (output.RemnantType_1==6) &\
        (output.Semimajor.notnull())

# apply the mask
output=output[idxBHBH]

In [ ]:
#EVOLVED files processing

#extracting the alpha parameter from the path of the file 
alpha = float(re.findall(r".+(?<=A)(.*)(?=L)",
                         paths[1])[0])

#read the columns we are interested in from the evolved file
evolved = pd.read_table(paths[1],                               # read file
                        sep='\s+',                              # separate by spaces
                        usecols=evolved_column_to_read,         # read only some columns
                        dtype=dict(zip(evolved_column_to_read,  # specify the types
                                       evolved_column_type)))   #
#NB: sep='\s+' is needed because there are different
#number of spaces separareting the columns

#adding the column with the alpha parameter
evolved['alpha'] = alpha

In [ ]:
#LOGFILE files processing

logfile = pd.read_csv(paths[2],    # read the file
                      header=None) # there is no header

In [ ]:
#Running Regex on the line of the logfile to extrac useful informations
df_RLO = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);RLO_BEGIN;").\  # searching for string "RLO_BEGIN"
            dropna().\                                                    # dropping nan
            rename(columns={0:'name', 1:'ID'}).\                          # rename columns
            groupby(['name']).\                                           # grouping by name
            size().\                                                      # and counting the number of RLO
            to_frame(name='RLO').\                                        # 
            reset_index()                                                 # to have a nice dataframe


df_CE = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);CE;").\  # searching for string "CE"
            dropna().\                                            # dropping nan
            rename(columns={0:'name', 1:'ID'}).\                  # rename
            groupby(['name']).\                                   # grouping by name
            size().\                                              # 
            to_frame(name='CE').\                                 # and counting the number of CE
            reset_index()                                         # to have a nice dataframe


df_BSN = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);BSN;").\  #searching for string "BSN"
            dropna().\                                              # dropping nan
            rename(columns={0:'name', 1:'ID'}).\                    #rename
            groupby(['name']).\                                     #grouping by name
            size().\                                                #
            to_frame(name='BSN').\                                  #and counting the number of BSN
            reset_index()                                           #to have a nice dataframe

df_No_Kick = logfile[0].str.extract(r"S;((?:\d*\_)?\d+);(\d+);SN;.+:(0):.+:.+:.+.").\ #searching for string "No_Kick"
            dropna().\                                                                # dropping nan
            rename(columns={0:'name', 1:'ID', 2: 'No_Kick'}).\                        #rename
            groupby(['name']).\                                                       #grouping by name
            size().\                                                                  #
            to_frame(name='No_Kick').\                                                #and counting the number of "No_Kick"
            reset_index()                                                             #to have a nice dataframe

In [ ]:
#MERGE
bhbh = evolved.merge(output, on=['name'], how='inner').\    #innerg join on the name between evolved and output
               merge(df_RLO, on=['name'], how='left').\     #left join on the name with df_RLO
               merge(df_CE,  on=['name'], how='left').\     #left join on the name with df_CE
               merge(df_BSN, on=['name'], how='left').\     #left join on the name with df_BSN
               merge(df_No_Kick, on=['name'], how='left').\ #final join on the name with No_Kicks
               fillna(value=0).\                            #setting nan to zero
               drop(columns=drop_list)                      #dropping no longer useful columms

In [ ]:
#Adding some columns with physical meaning
bhbh['tdelay'] = bhbh['GWtime'] + bhbh['BWorldtime'] #time delay

#defining the max mass of output
bhbh['Mass_max_out'] = bhbh['Mass_1_out']
bhbh['Mass_max_out'] = bhbh['Mass_max_out'].\
                        where(cond=(bhbh['Mass_max_out'] > bhbh['Mass_0_out']),
                              other=bhbh['Mass_0_out'])

#defining q=m1/m2 with m2>,m1
bhbh['q'] = bhbh['Mass_1_out']/bhbh['Mass_0_out']
bhbh['q'] = bhbh['q'].\
            where(cond=(bhbh['Mass_1_out'] < bhbh['Mass_0_out']),
                  other=bhbh['Mass_0_out']/bhbh['Mass_1_out'])

#defining the Chirp mass
bhbh['Mass_chirp'] = ((bhbh['Mass_0_out'] * bhbh['Mass_1_out'])**(3/5))/((bhbh['Mass_0_out'] + bhbh['Mass_1_out'])**(1/5))

We apply the above function with Dask to the whole dataset via the Bag objects.

In [ ]:
#bag of lists generated by a list comprehension
bag=db.from_sequence([
            [dir_ + f'/0/output_{thread}.csv', 
             dir_ + f'/0/evolved_{thread}.dat',
             dir_ + f'/0/logfile_{thread}.dat'] for dir_ in dir_list for thread in range(30)],
             npartitions=30*60) #the number of partitions is set as such
                                #in order to have more or less one list per partition 

In [ ]:
def preprocessing_bag_of_thread(paths):
    '''
    The missing code from the function above 
    '''
    return bhbh #a pandas dataframe

In [ ]:
bag_of_df = bag.map(preprocessing_bag_of_thread) #Map the preprocessing function to the bag
bag_of_dicts = bag_of_df.map(lambda df: df.to_dict(orient='records'))\
                        .flatten() #force a mapping of the bag to dictionaries in order to use the .to_dataframe() method
bhbh = bag_of_dicts.to_dataframe() #delayed function 
bhbh = bhbh.compute() #actual compute that returns the final pandas DataFrame

# A recap on Machine Learning approaches used

## Neural Networks

The first approach we used in order to tackle the task was to build a neural network (NN) in order to predict the mass ratio and the chirp mass, so that we could infer their distributions. 
We opted for this approach due to its flexibility, since the architecture of the network can be adapted and tailored for what is needed, from changing its depth to the activation functions used for each layer, the number of nodes and the objective of the task.
In the following sections are presented how the task was modeled and what kind of architectures we tried to use in order to predict the values for the mass ratios and the chirp masses. 

## Regression Vs Classification
Since the quantities of which we want to learn the distribution are continous, the problem can be modeled in different ways depending on how we want to deal with them:
* Keeping its continous nature, the task can be formulated as a **regression** where the algorithm, starting from the initial features of each record, tries to predict the exact value of the final quantity as close as possible by approximating the complex dependencies from the input values. This task can be become extremely computationally heavy as the complexity of the data increases. This is due to the size of the network needed to capture such intricate relatioships;
* A different approach in dealing with continous target values is to discretize them by assigning different labels to different ranges of their values. This allows to characterize each data record with its own label. In this case the problem can be tackled by a **classification** algorithm which will try to predict the label instead of the exact value of the discretized quantity. As the number of labels increases, the complexity of the task grows.

## Direct Prediction Vs SoftMax Approach

In trying to predict the correct label in a classification algorithm its `objective` can be modified in order to gain more insights on what the model is learning. By objective is intended the way the output of the task is presented.
In this case we will refer to a direct prediction when the output consists in just the label infered from the initial input values, while an alternative approach can be returing the probability associated with each label to be assigned to a praticular record.
This is achived by using as the output activation function the `Softmax` function:
<center>
<div style='background-color:#f7f7f7; text-align: center; max-width: 250px; max-height: 50px; padding-top:30px; padding-left:20px; padding-right:20px; padding-bottom:30px; '>$$
\sigma(z)_j  = \frac{e^{z_j}}{\sum_{k} e^{z_k}}
$$</div></center>
Here for each output node value $z_j$, the number of which is equal to the number of possible labels, is associated the probability $\sigma(z)_j $ of being guessed for the particular record used as input of the NN.
By inspecting the resulting probability distributions several quantities can be infered such as the most probable label as well the dispersion of the distribution which can give insights on how well the model is learning.

## Tested Architectures
For each of the following proposed architecture several variants have been tested in a grid search fashion, in order to improve the capabilities of the algorithms.
Due to the extreme complexity of the data neither regression-like tasks, nor the classification ones yielded satisfying results. Every NN has been implemented by the `PyTorch` python package.

### Regression Architecture
The network is designed with fully connected layers that converge into a single output node.
In a convolutional network fashion, the algorithm present parallel layers disconnected one from another so that each distinct group can focus on learning different aspects of the input data, for later being recombined into a single value.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.in1 = nn.Linear(6,4)
        self.in2 = nn.Linear(6,4)
        self.in3 = nn.Linear(6,4)
        # self.in4 = nn.Linear(6,4)

        self.lin1_1 = nn.Linear(4,4)
        self.lin1_2 = nn.Linear(4,4)
        self.lin1_3 = nn.Linear(4,4)
        
        self.out = nn.Linear(12,1)

    # x represents our data
    def forward(self, x):
        x1 = self.in1(x)
        x1 = torch.relu(x1)
        
        x1 = self.lin1_1(x1)
        x1 = torch.relu(x1)
        
        x2 = self.in2(x)
        x2 = torch.relu(x2)
        
        x2 = self.lin1_2(x2)
        x2 = torch.relu(x2)
        
        x3 = self.in3(x)
        x3 = torch.relu(x3)
        
        x3 = self.lin1_3(x3)
        x3 = torch.relu(x3)
        
        
        output = torch.concat((x1, x2, x3), axis=1)
        output = self.out(output)
        output = torch.relu(output)
        
        return output

<!--center style="margin-left: 10%; margin-right: 10%; background-color: #eeeeee; padding-top: 10px; padding-bottom: 10px;"><figure><img src="Figures/lcp/NN.png" width="60%" height="30%"><figcaption></figcaption></figure></center><br-->

<center>
<table><tbody><tr>
<td><img src="Figures/lcp/NN.png"></td>
<td width=50%> <img src="Figures/lcp/regression_distribution.png" ></td>
</tr></tbody>
    <tfoot><td>How a Neural Network is built with regression</td><td>An example of the NN output</td></tfoot>
</table>
</center><br>

<!--center style="margin-left: 10%; margin-right: 10%; background-color: #eeeeee; padding-top: 10px; padding-bottom: 10px;"><figure><img src="Figures/lcp/regression_distribution.png" width="100%" height="30%"><figcaption></figcaption></figure></center><br-->

### Softmax Architecture
The NN is made of fully connected layers of increasing size, which correspond to the number of nodes they are made of.
In the final layer, for each output node is computed the softmax function which yields the probability distribution of predicting the label for a given input.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.input = nn.Linear(5, 10)
        self.linear1 = nn.Linear(10, 15)
        self.output = nn.Linear(15, 20)

    # x represents our data
    def forward(self, x):
        x = self.input(x)
        x = torch.relu(x)
        
        x = self.linear1(x)
        x = torch.relu(x)
        
        x = self.output(x)
        x = F.softmax(x, dim=0)
        
        return x

<center>
<table><tbody><tr>
<td width=50%><img src="Figures/lcp/NNsm.png"></td>
<td> <img src="Figures/lcp/sm_distribution.png" ></td>
</tr></tbody>
    <tfoot><td>How a Neural Network is built with classification</td><td>An example of the NN output</td></tfoot>
</table>
</center><br>

# XGBoost

Our final choice of algorithm to implement is the `XGBoost` classifier. 
XGBoost is a python library that is optimized for distributed gradient boosting designed to be efficient and flexible.
In particular, XGBoost is used in Supervised Learning frameworks for different tasks: for example classification or regression.
It provides a parallell tree boosting that can be used to tackle our problem of predicting the mass distribution given the initial conditions of a binary system.

The goal of our analysis is to develop an algorithm that takes as input some features of the binary system (namely the intial masses of the stars, the metallicity, the $\alpha$ parameter, the eccentricity and the semimajor-axis) and returns the probability distribution of a chosen feature (for example $q$ or the $\mathcal{M}$).
Such algorithm can be obtained specifying as `objective` the `multi:softprob` as parameter of the XGBoost.
As anticipated in the previous section, we discretized the $q$ or the $\mathcal{M}$ so as to the resulting training is aimed at a multiclass calssification (`multi`) that returns for each record the probability of being in each class (`softprob`).
The other parameters have been specified based on the results of a grid search.

In the following cell we report the code that defines the parameters of the algorithm.

In [ ]:
# Parameters
max_depth = 0              #maximum depth of a tree
learning_rate = 0.2        #learning rate for optimization algorithm
grow_policy = 'lossguide'  #controls the way new nodes are added; one of the possible choices
n_estimators = 5           #equivalente to num_boosted_rounds(); Get number of boosted rounds
max_leaves = 400           #maximum number of nodes to be added

param = {'objective': 'multi:softprob', #classificatio that returns probabilities
         'tree_method': 'hist',         #one of the possible choices
         'n_estimators' : n_estimators,
         'grow_policy' : grow_policy, 
         'max_depth' : max_depth,
         'learning_rate' : learning_rate,
         'max_leaves' : max_leaves,
         'num_class': n_bins,           #number of classes
         'n_jobs' : -1,                 #for parallel processing
        }

In the following cell we create a DaskDMatrix which is the data structure used for training by the dask implementation of XGBoost. 

In [ ]:
dtrain = xgb.dask.DaskDMatrix(client, features, label=labels, weight=weights)

The `weights` parameter can be added to deal with unbalanced datasets, e.g. a dataset with different occupancies for each class.
The default way of assigning weights is by using the reciprocal of the frequency of each bin.
Additionaly, we define another function giving more importance to the central labels as shown below.

In [ ]:
countings = bhbh_train['label'].to_frame().groupby('label').size().compute()
total_counts = countings.sum()

#Assign to a new column
def assign_weight_custom(i):
    return np.log(total_counts/(n_bins*countings[i])+1) 

def assign_weight_default(i):
    return total_counts/(n_bins*countings[i]) 

bhbh_train['weight'] = bhbh_train['label'].map(assign_weight_*) # depending on the implementation we can choose _custom _default

<center><figure><img src="Figures/lcp/weights_vs_labels.png" ><figcaption>This is a simple plot showing the weights as a function of the labels.</figcaption></figure></center>

For our final algorithm we have found the custom weights to be more effective.

Then we proceed with the training of the XGBoost with the chosen parameters and the given training set.

In [ ]:
# train XGB
bst = xgb.dask.train(client, param, dtrain, evals=[(dtrain, "train")])

When the model is trained, we apply it to the test dataset to obtain some predictions on new data.
In particular the algorithm will return the probability distribution of each record of belonging to each class. 
To predict a particular label we can then extract the class that has the highest probability and confront it with the true label.

In [ ]:
## distributions of the probability of each class for each record
output_test = xgb.dask.predict(client, bst, features_test).persist() 

## Predicition of the label with Maximum estimation
predicted_label = output_test.idxmax(axis=1).persist()

## Computing the true labels and the predicted

#true labels
Y_test = label_test.values.compute()
#predicted labels
Y_predicted = predicted_label.values.compute()

These are some of the probability distributions coming out of the XGBoost algorithm that we display to give a general idea of their shapes.
<center>
<table><tbody><tr>
<td><img src="Figures/lcp/prob_dist_of_labels.png"></td>
<td> <img src="Figures/lcp/prob_dist_of_labels_peaked.png" ></td>
</tr></tbody>
    <tfoot><td>An example of some probability distributions</td><td>Another example of more peaked distributions</td></tfoot>
</table>
</center><br>

## Grid search to find the best hyperparameters

In order to determine the best hyperparameters for the model we implement a grid search on different values of the `max_depth`, the `learning_rate`, the `n_estimators` and `max_leaves`.

The grid search has been done by creating a list of possible values for each parameter and cyclying with for loops over them, testing each combination. 

In order to compare different results we have introduced some metrics, like the Kullback-Leibler, the Jensen-Shannon divergence and an accuracy measure defined by us.

#### Kullback-Leibler

The Kullback-Leibler divergence (also called relative entropy) is a type of statistical distance that measures the distance between two distributions $p$ and $q$. 
It is defined as:
<center>
<div style="background-color:#f7f7f7; text-align: center; max-width: 300px; max-height: 50px; padding-top:30px; padding-left:20px; padding-right:20px; padding-bottom:30px;">
$$ D_{KL}(p||q)=\sum_i p_i \cdot \log\left(\frac{p_i}{q_i}\right) $$
    </div></center>

In particular, $D_{KL}=0$ if the two distributions are identical. It is already implemented in the `scipy` library.

In [ ]:
KL_div = scipy.stats.entropy(counts_predicted, counts_test)

#### Jensen-Shannon divergence

The Jensen-Shannon divergence is another type of statistical distance, based on the Kulback-Leibler with the difference that it is symmetric and has a finite value (while the KL divergence does not have an upper limit).

It is defined as:

<center>
<div style='background-color:#f7f7f7; text-align: center; max-width: 300px; max-height: 50px; padding-top:30px; padding-left:20px; padding-right:20px; padding-bottom:30px; '>
$$ D_{JS}(p||q)=   \frac{1}{2}D_{KL}(p||m)+ \frac{1}{2} D_{KL}(q||m)$$
</div>
</center>

where $m={\frac {1}{2}}(p+q)$. It is also already implemented in the `scipy` library.

In [ ]:
from scipy.spatial.distance import jensenshannon as JS
JS_div = JS(counts_test, counts_predicted, base=2)

In the grid search, the above defined measures have been used to compare the true distributions of the $q$ and the $\mathcal{M}$ of the test set and the one predicted by our algorithm.
It is difficult to interpret these measures by themeselves because we do not know if a given distance is a good result or not. We can anyway use these measures to compare different models in the grid search by selecting the one that minimizes the distances.

#### A threshold accuracy measurement

The typical measure of accuracy of classification task consists in calculating the percentange of correctly predicted labels is not very informative in our problem.
We are actually dealing with a pretty large number of classes ($100$) and therefore we would like to consider the fact that if we predict a label that is near the true one we are not completely unsatisfied.
For this reason we define an accuracy that counts the percentange of corrected labels within a tollerance defined by a threshold. 
This accuracy is useful to understand what is the percentange of labels that we correctly predict within a given tollerance.

In [ ]:
## Defining a prediction accurate if not further from the true of a value of threshold
def accuracy_threshold(Y_test, Y_predicted, threshold = 5):
    out=np.abs(Y_test-Y_predicted)
    mask = out < threshold
    return np.sum(mask)/len(Y_test)

We can now finaly present the code that we have written in order to run the grid search, based on nested for loops that cycles on the list of parameters.

In [ ]:
#parameters lists
max_depth_list = [4, 6, 8, 10, 0]
learning_rate_list = [0.1, 0.2, 0.3]
grow_policy = 'lossguide'
n_estimators_list = [5, 15, 20]
max_leave_list = [50, 100, 200, 400]

In [ ]:
#GRID SEARCH

#file in wich to save the results of the grid search
f = open('grid_search_results.csv', 'a')
writer = csv.writer(f)

#writing the head of the file
head = ['max_depth', 'learning_rate', 'n_estimators', 'max_leaves', 'KL_div', 'JS_div', 'AT']
writer.writerow(head)

#and printing to std output
print('max_depth', 'learning_rate', 'n_estimators', 'max_leaves', 'KL_div', 'JS_div', 'AT')


for max_depth in max_depth_list:
    for learning_rate in learning_rate_list:
        for n_estimators in n_estimators_list:
            for max_leaves in max_leave_list:
                
                #selected parameters
                param = {'objective': 'multi:softprob',
                         'tree_method': 'hist', 
                         'n_estimators' : n_estimators, #equivalent to num_boost_round
                         'grow_policy' : grow_policy, 
                         'max_depth' : max_depth,
                         'learning_rate' : learning_rate,
                         'max_leaves' : max_leaves,
                         'num_class': n_bins,
                         'n_jobs' : -1,
                        }
                
                #training the model
                bst = xgb.dask.train(client, param, dtrain, evals=[(dtrain, "train")])
                
                #extracting the predictions
                output_test = xgb.dask.predict(client, bst, features_test).persist() #already a dataframe
                predicted_label = output_test.idxmax(axis=1).persist()
                Y_test = label_test.values.compute()
                Y_predicted = predicted_label.values.compute()
                
                #Kulback-Leibler
                counts_test, _ = np.histogram(Y_test, density=True, bins=n_bins)
                counts_predicted, _ = np.histogram(Y_predicted, density=True, bins=n_bins)
                
                KL_div = scipy.stats.entropy(counts_predicted, counts_test)
                
                ## Jensen-Shannon divergence
                JS_div = JS(counts_test, counts_predicted, base=2)
                
                #Accuracy threshold
                AT = accuracy_threshold(Y_test, Y_predicted, 15)
                
                #saving the results in a csv file
                row = [max_depth, learning_rate, n_estimators, max_leaves, KL_div, JS_div, AT] 
                writer.writerow(row)
                
                #printing to std output
                print(max_depth, learning_rate, n_estimators, max_leaves, KL_div, JS_div, AT)  

#closing the file
f.close()              

The results have been finally saved in a `csv` file. 
With all the three tested learning rates we obtain the same results, meaning the all the three values reach convergence. 
We have then chosen as best parameter `learning_rate =0.2`. Also the number of estimators does not affect the results, we have then kept the lowest one, `n_estimator = 5`. 
For the maximum leaves we obtain as best parameter `max_leaves= 400` and as expected, the perfomances improves by increasing the maximum depth and the best parameter is `max_depth=0` (which means that the algoritms stops only when it reaches convergece).

In the following table we report the best parameters. These parameters are the ones that we use for the rest of the analysis.




Parameter | Value
---|:---: 
max_depth| 0
learning rate|0.2
number of estimators| 5
maximum leaves | 400

# Results
The algorithm is able to perform predictions either on the `q` or the `chirp mass`, as its implementation barely varies.

Here we are showing the results of the predictions made with `q`. In the following section we are showing the same results but run with the `chirp mass` 

## Analysis with the `Mass Ratio` $q$

## Feature importance
A first analysis of the performance of the model is carried out through the feature importance.
This analysis shows how much each of the features of the training dataset is used to split the data in the XGBoost architecure.
Other similar importance metrics such as the gain, which is the average of the gain per split when a specific feature was used, show that `Mass_0`, `Z_0` the metallicity and `Mass_1` were the most useful and could better help the training process.
We remind the feature importance does not take into account the test set, as it is just another way of displaying the information learnt dirung the training.

<!--center style="margin-left: 5%; margin-right: 5%; background-color: #eeeeee; padding-top: 10px; padding-bottom: 10px;"-->
<center>
<table><tbody><tr>
<td> <img src="Figures/lcp/importance_q_yeskicks_weight.png"></td>
<td> <img src="Figures/lcp/importance_q_yeskicks_gain.png" ></td>
<td><img src="Figures/lcp/importance_q_yeskicks_total_gain.png"> </td>
</tr></tbody>
    <tfoot>Importance fo each feature with different scorings</tfoot>
</table>
</center><br>

Our following analysis distinguishes the systems that do not display any kicks in the binary evolution.
To get these results we run the whole training and prediction on a subset of systems, namely only those characterized by the flag `NoKicks=2`. 
We make this distinction because we are trying to understand how the random component introduced by the kicks alters the accuracy of the predictions.
However for the previous graphs the results coming out of these two approaches show negligible differences.

## Confusion matrix
To check by eye how the algorithm performs on the unseen test set, we plot the confusion matrix of the predictions.

<center>
<table><tbody><tr>
<td><img src="Figures/lcp/confusion_matrix_labels_yeskicks.png"></td>
<td> <img src="Figures/lcp/confusion_matrix_labels_nokicks.png" ></td>
</tr></tbody>
    <tfoot><td>All systems</td><td>Nokick</td></tfoot>
</table>
</center><br>

Each square of the picture shows the number of counts that were assigned to a specific label, given the value of the true label.
It is immediate to spot higher countings of binary systems that are in accordance with their true label, thus suggesting that the model really understood the hidden processes.

Finally it is easy to spot how higher labels tend to be more frequently misclassified, but this is easily explained by the unbalanced dataset we started with.
Very few binary systems of actual lower q labels happen to be missclassified with high labels as shown by the white patch in the upper left corner.

## Comparison of the empirical and predicted `q` label distributions
A different and more comprehensive visualization to better check the performances of the model is to plot the distributions of the empirical and true labels, as shown in the following plots.

<center>
<table><tbody><tr>
<td><img src="Figures/lcp/hist_q_yes_kicks.png"></td>
<td> <img src="Figures/lcp/hist_q_nokicks.png" ></td>
</tr></tbody>
    <tfoot><td>All systems</td><td>Nokick</td></tfoot>
</table>
</center><br>


The distributions seem to match very well, in all the various regions of the q labels.
For the whole dataset, the model seems to capture, but only up to a certain extent, the bump around the region of labels 60 to 70 as there are some deviations from the smooth trend.
There seems to be a slight overcouting for the smaller labels compared to the true distribution, but as can be seen from the Kullback-Liebler and Jensen-Shannon metrics, this does not influence the overall correspondance between the two distributions.
However, for the systems without the kicks, the bump is absent and so is for the predicted distribution.
As can be seen from the comparison of the metrics, the second predicted distribution is more closely matching to the empirical data.

Additionally we plot the so called $\Delta q$ which measures how much and how often labels were overestimated or underestimated.
We obtain a 95% confidence interval of $\Delta q$ ranging from -43 to 24 for the whole dataset and from -31 to 15 for the nokicks subset. Furthermore, this observation suggests that our algorithm is sensible to the stochasticity of the binary evolution.

To better display the way labels are missassigned we plot in the inset the logarithmic distributions which shows an asymmetry to the left: labels are more prone to be underestimated.

<center>
<table><tbody><tr>
<td><img src="Figures/lcp/hist_delta_q_yeskicks.png"></td>
<td> <img src="Figures/lcp/hist_delta_q_nokicks.png" ></td>
</tr></tbody>
    <tfoot><td>All systems</td><td>Nokick</td></tfoot>
</table>
</center><br>

More specifically, within an error of $\Delta q=25$ in the assignement of the labels, we have an accuracy of 90% for the whole dataset, whereas the same accuracy for the nokicks dataset is achieved with a $\Delta q=13$.   

## Maximum probabilities
The XGBoost algorithm outputs a discrete probability distribution for each possible label. To make the final prediction for each system, we focus on the label with maximum probability. 

With this approach in mind it is quite complex to check all the distribution shapes for all the systems.
Since we are interested in the full picture described by our output we decide to plot the two higher probabilities (#1 and #2 in the following graphs), neglecting the labels to which they are associated. 
This approach should enable us to somehow check the shape of the probability distributions.

<center>
<table><tbody><tr>
<td><img src="Figures/lcp/tops_q_yeskicks.png"></td>
<td> <img src="Figures/lcp/tops_q_nokicks.png" ></td>
</tr></tbody>
    <tfoot><td>All systems</td><td>Nokick</td></tfoot>
</table>
</center><br>

For simplicity of notation we are going to define the following to names:
- **peaked distribution** region where #1>0.8 where we expect to see some peaked distributions
- **flat distribution** region where #1<0.2 where we expect to see quite flat distributions

## A deeper focus on the probabilities and some specific Z
In a more specific analysis we plot the **peaked distribution** region for diffierent metallicity values Z.  
Surprisingly, we observe an higher density of systems in this region for Z=0.017 and Z=0.06, as shown below. For comparison we also plot the same ragion for systems of another metallicity which do not display such high densities. 

Notice the title of each graph below: for a specific metallicity, in this region of #1>0.8, the fracion of systems being predicted with $\Delta q$=0 is always higher than 85%. 

<center>
<table><tbody><tr>
<td><img src="Figures/lcp/zoom_peakregion_q_yeskicks.png"></td></tr><tr><td>All systems</td></tr><tr>
<td> <img src="Figures/lcp/zoom_peakregion_q_nokicks.png" ></td></tr><tr><td>Nokick</td></tr><tr>
</tr></tbody>
</table>
</center><br>

Here is a summary of the guessed systems, separated by the datasets used:

<center style="background-color: #f4f4f4; margin-left:10%; margin-right:10%"><b>q</b></center>

|Quantity| Fraction of systems|
|---:|:---|
|Fraction of data in the flat region| 0.82|
|Fraction of exactly guessed labels| 0.23|
|Exactly guessed label in flat region / total points in flat region|0.16|
|Exactly guessed label in flat region / total exactly flat labels|0.58|

<center style="background-color: #f4f4f4; margin-left:10%; margin-right:10%"><b>q Nokick</b></center>


|Quantity| Fraction of systems|
|---:|:---|
|Fraction of data in the flat region| 0.72|
|Fraction of exactly guessed labels| 0.30|
|Exactly guessed label in flat region / total points in flat region|0.20|
|Exactly guessed label in flat region / total exactly flat labels|0.47|

## Distributions of #1,#2 depending on $\Delta q$

### Perfectly predicted $\Delta q=0$
To further investigate if the perfectly predicted systems are associated to a specific region of the space #1,#2 and thus highlighting any features in the probability distribution, we display #1 and #2 for the systems $\Delta q=0$.

A reasonable expectation is that distributions with a single high peak are perfecly predicted as the algorithm is sure about the guess.
This claim would implay an higher density of points in the peaked region and a scarcity of points in rest of the plane.

As shown in the following plots, this is not true and we observe no particular trends or clusterings of points, compared to the overall distribution with any value of $\Delta q$.
<center>
<table><tbody><tr>
<td><img src="Figures/lcp/tops_q_perfect_yeskicks.png"></td>
<td> <img src="Figures/lcp/tops_q_perfect_nokicks.png" ></td>
</tr></tbody>
    <tfoot><td>All systems</td><td>Nokick</td></tfoot>
</table>
</center><br>

### Wrongly predicted $\Delta q> 25$ (all systems) or $13$ (no kick)
On the other hand, we display #1 and #2 of the systems that correspont to the worst 10% predictions. These systems are found to be those with a $\Delta q>25$ for the whole dataset or $\Delta q>13$ for the nokick subset.

Given the previous analysis we expect an uniform distribution of points. However this time we find that the majority of these kind of systems if found in the flat region. 
This is a reasonable result as these systems display an almost flat probability distribution, implying that the maximum value of the probability distribution is not able to characterize the true label.

<center>
<table><tbody><tr>
<td><img src="Figures/lcp/tops_q_error_more_25_yeskicks.png"></td>
<td> <img src="Figures/lcp/tops_q_error_more_13_nokicks.png" ></td>
</tr></tbody>
    <tfoot><td>All systems</td><td>Nokick</td></tfoot>
</table>
</center><br>

## Characterizing the dependencies on the input parameters for $\Delta q=0$ and $\Delta q>25$ (all systems) or $13$ (no kick)

### Z comparison
Each plot shows the fraction of binaries for every metallicity. 
In the top row we display the systems with $\Delta q>25$ or 13, depending whether or not the kicks were included.
In the last row are displayed the systems with $\Delta q=0$.

As we can see, for the first row, after a certain value of Z there are some deviations from the trend of the whole dataset, as shown in darker blue. 

In the first row, we show the distribution of the whole dataset compared to the systems calssified with an accuracy od $\Delta q>25$ or $13$. In the second row we focus the compairson to the systems with $\Delta q=0$.

Qualitatively speaking, there are some differences on the distributions, especially on the tails and for some values of Z.

<center>
<table>
    <tr>
        <td><img src="Figures/lcp/z_comparison_q_yeskicks.png"width="100%" height="30%"></td>
        <td> <img src="Figures/lcp/z_comparison_q_nokicks.png"width="100%" height="30%"></td>
    </tr>
    <tr>
        <td>All systems</td>
        <td>Nokick</td>
    </tr>
    <tr>
        <td><img src="Figures/lcp/z_comparison_q_one_peak_yeskicks.png"></td>
        <td> <img src="Figures/lcp/z_comparison_q_one_peak_nokicks.png" ></td>
    </tr>
    <tr>
        <td>All systems</td>
        <td>Nokick</td>
    </tr>
</table>
</center><br>

### Semimajor axis
Each plot shows the distribution of the binaries depending on the semimajor axis a. 
In the top row we display the systems with $\Delta q>25$ or 13, depending whether or not the kicks were included.
In the last row are displayed the systems with $\Delta q=0$.

No significant deviation are found.

<center>
<table>
    <tr>
        <td><img src="Figures/lcp/a_comparison_q_yeskicks.png"width="100%" height="30%"></td>
        <td> <img src="Figures/lcp/a_comparison_q_nokicks.png"width="100%" height="30%"></td>
    </tr>
    <tr>
        <td>All systems</td>
        <td>Nokick</td>
    </tr>
    <tr>
        <td><img src="Figures/lcp/a_comparison_q_one_peak_yeskicks.png"></td>
        <td> <img src="Figures/lcp/a_comparison_q_one_peak_nokicks.png" ></td>
    </tr>
    <tr>
        <td>All systems</td>
        <td>Nokick</td>
    </tr>
</table>
</center><br>


### Eccentricity 
Each plot shows the distribution of the binaries depending on the eccentricity e. 
In the top row we display the systems with $\Delta q>25$ or 13, depending whether or not the kicks were included.
In the last row are displayed the systems with $\Delta q=0$.

No significant deviations are found.

<center>
<table>
    <tr>
        <td><img src="Figures/lcp/e_comparison_q_yeskicks.png"width="100%" height="30%"></td>
        <td> <img src="Figures/lcp/e_comparison_q_nokicks.png"width="100%" height="30%"></td>
    </tr>
    <tr>
        <td>All systems</td>
        <td>Nokick</td>
    </tr>
    <tr>
        <td><img src="Figures/lcp/e_comparison_q_one_peak_yeskicks.png"></td>
        <td> <img src="Figures/lcp/e_comparison_q_one_peak_nokicks.png" ></td>
    </tr>
    <tr>
        <td>All systems</td>
        <td>Nokick</td>
    </tr>
</table>
</center><br>


## Mass dependencies

### Distribution of the input masses 
We show the distribution of the input masses as a references for the further discussion. 

<center><figure><img src="Figures/lcp/mass_0_mass_1.png" width="40%"><figcaption></figcaption></figure></center><br>

### Characterizing the distribution of the initial masses for systems with $\Delta q=0$ and $\Delta q>25$ (all systems) or $13$ (no kick)

Each plot shows the distribution of the binaries masses. 
In the top row we display the systems with $\Delta q>25$ or 13, depending whether or not the kicks were included.
In the last row are displayed the systems with $\Delta q=0$.

No significant deviations are found.

<center>
<table>
    <tr>
        <td><img src="Figures/lcp/mass_0_mass_1_error_25_q_yeskicks.png"width="100%" height="30%"></td>
        <td> <img src="Figures/lcp/mass_0_mass_1_error_13_q_nokicks.png"width="100%" height="30%"></td>
    </tr>
    <tr>
        <td>All systems and $\Delta q$>25</td>
        <td>Nokick and $\Delta q$>13</td>
    </tr>
    <tr>
        <td><img src="Figures/lcp/mass_0_mass_1_all_right_yeskicks.png"></td>
        <td> <img src="Figures/lcp/mass_0_mass_1_all_right_nokicks.png" ></td>
    </tr>
    <tr>
        <td>All systems and $\Delta q=0$</td>
        <td>Nokick and $\Delta q=0$</td>
    </tr>
</table>
</center><br>


## Analysis with the `chirp mass` $\mathcal{M}$
This last section is devoted to the same analysis run above but on the `chirp mass` $\mathcal{M}$ instead of $q$

## Feature importance

<center>
<table><tbody><tr>
<td> <img src="Figures/lcp/importance_Chirp_Mass_yeskicks_weight.png"></td>
<td> <img src="Figures/lcp/importance_Chirp_Mass_yeskicks_gain.png" ></td>
<td><img src="Figures/lcp/importance_Chirp_Mass_yeskicks_total_gain.png"> </td>
</tr></tbody>
    <tfoot>Importance fo each feature with different scorings</tfoot>
</table>
</center><br>

## Confusion matrix

<center>
<table><tbody><tr>
<td><img src="Figures/lcp/confusion_matrix_labels_Chirp_Mass_yeskicks.png"></td>
<td> <img src="Figures/lcp/confusion_matrix_labels_Chirp_Mass_nokick.png" ></td>
</tr></tbody>
    <tfoot><td>All systems</td><td>Nokick</td></tfoot>
</table>
</center><br>

## Comparison of the empirical and predicted $\mathcal{M}$ label distributions

<center>
<table><tbody><tr>
<td><img src="Figures/lcp/hist_Chirp_Mass_yeskicks.png"></td>
<td> <img src="Figures/lcp/hist_Chirp_Mass_nokicks.png" ></td>
</tr></tbody>
    <tfoot><td>All systems</td><td>Nokick</td></tfoot>
</table>
</center><br>



<center>
<table><tbody><tr>
<td><img src="Figures/lcp/hist_delta_Chirp_Mass_yeskicks.png"></td>
<td> <img src="Figures/lcp/hist_delta_Chirp_Mass_nokicks.png" ></td>
</tr></tbody>
    <tfoot><td>All systems</td><td>Nokick</td></tfoot>
</table>
</center><br>

## Maximum probabilities

<center>
<table><tbody><tr>
<td><img src="Figures/lcp/tops_Chirp_Mass_yeskicks.png"></td>
<td> <img src="Figures/lcp/tops_Chirp_Mass_nokicks.png" ></td>
</tr></tbody>
    <tfoot><td>All systems</td><td>Nokick</td></tfoot>
</table>
</center><br>

## A deeper focus on the probabilities and some specific Z

<center>
<table><tbody><tr>
<td><img src="Figures/lcp/zoom_peakregion_Chirp_Mass_yeskicks.png"></td></tr><tr><td>All systems</td></tr><tr>
<td> <img src="Figures/lcp/zoom_peakregion_Chirp_Mass_nokicks.png" ></td></tr><tr><td>Nokick</td></tr><tr>
</tr></tbody>
</table>
</center><br>

Here is a summary of the guessed systems, separated by the datasets used

<center style="background-color: #f4f4f4; margin-left:10%; margin-right:10%"><b>Chirp Mass</b></center>


|Quantity| Fraction of systems|
|---:|:---|
|Fraction of data in the flat region| 0.25|
|Fraction of exactly guessed labels| 0.53|
|Exactly guessed label in flat region / total points in flat region|0.24|
|Exactly guessed label in flat region / total exactly labels|0.11|


<center style="background-color: #f4f4f4; margin-left:10%; margin-right:10%"><b>Chirp Mass Nokick</b></center>

|Quantity| Fraction of systems|
|---:|:---|
|Fraction of data in the flat region| 0.23|
|Fraction of exactly guessed labels| 0.59|
|Exactly guessed label in flat region / total points in flat region |0.25|
|Exactly guessed label in flat region / total exactly labels |0.10|

## Distributions of #1,#2 depending on $\Delta \mathcal{M}$

### Perfectly predicted $\Delta \mathcal{M}=0$

<center>
<table><tbody><tr>
<td><img src="Figures/lcp/tops_Mass_Chirp_perfect_yeskicks.png"></td>
<td> <img src="Figures/lcp/tops_Mass_Chirp_perfect_nokicks.png" ></td>
</tr></tbody>
    <tfoot><td>All systems</td><td>Nokick</td></tfoot>
</table>
</center><br>

### Wrongly predicted $\Delta \mathcal{M}> 5$ (all systems) or $4$ (no kick)

<center>
<table><tbody><tr>
<td><img src="Figures/lcp/tops_Chirp_Mass_error_more_5_yeskicks.png"></td>
<td> <img src="Figures/lcp/tops_Chirp_Mass_error_more_4_nokicks.png" ></td>
</tr></tbody>
    <tfoot><td>All systems</td><td>Nokick</td></tfoot>
</table>
</center><br>

## Characterizing the dependencies on the input parameters for $\Delta \mathcal{M}=0$ and $\Delta \mathcal{M}>5$ (all systems) or $4$ (no kick)

### Z comparison

<center>
<table>
    <tr>
        <td><img src="Figures/lcp/z_comparison_Chirp_Mass_yeskicks.png"width="100%" height="30%"></td>
        <td> <img src="Figures/lcp/z_comparison_Chirp_Mass_nokicks.png"width="100%" height="30%"></td>
    </tr>
    <tr>
        <td>All systems</td>
        <td>Nokick</td>
    </tr>
    <tr>
        <td><img src="Figures/lcp/z_comparison_Mass_Chirp_one_peak_yeskicks.png"></td>
        <td> <img src="Figures/lcp/z_comparison_Mass_Chirp_one_peak_nokicks.png" ></td>
    </tr>
    <tr>
        <td>All systems</td>
        <td>Nokick</td>
    </tr>
</table>
</center><br>

### Semimajor axis

<center>
<table>
    <tr>
        <td><img src="Figures/lcp/a_comparison_Mass_Chirp_yeskicks.png"width="100%" height="30%"></td>
        <td> <img src="Figures/lcp/a_comparison_Mass_Chirp_nokicks.png"width="100%" height="30%"></td>
    </tr>
    <tr>
        <td>All systems</td>
        <td>Nokick</td>
    </tr>
    <tr>
        <td><img src="Figures/lcp/a_comparison_Mass_Chirp_one_peak_yeskicks.png"></td>
        <td> <img src="Figures/lcp/a_comparison_Mass_Chirp_one_peak_nokicks.png" ></td>
    </tr>
    <tr>
        <td>All systems</td>
        <td>Nokick</td>
    </tr>
</table>
</center><br>


### Eccentricity 

<center>
<table>
    <tr>
        <td><img src="Figures/lcp/e_comparison_Chirp_Mass_yeskicks.png"width="100%" height="30%"></td>
        <td> <img src="Figures/lcp/e_comparison_Chirp_Mass_nokicks.png"width="100%" height="30%"></td>
    </tr>
    <tr>
        <td>All systems</td>
        <td>Nokick</td>
    </tr>
    <tr>
        <td><img src="Figures/lcp/e_comparison_Mass_Chirp_one_peak_yeskicks.png"></td>
        <td> <img src="Figures/lcp/e_comparison_Mass_Chirp_one_peak_nokicks.png" ></td>
    </tr>
    <tr>
        <td>All systems</td>
        <td>Nokick</td>
    </tr>
</table>
</center><br>

## Mass dependencies

### Characterizing the distribution of the initial masses for systems with $\Delta \mathcal{M}=0$ and $\Delta \mathcal{M}>5$ (all systems) or $4$ (no kick)

<center>
<table>
    <tr>
        <td><img src="Figures/lcp/mass_0_mass_1_error_5_Chirp_mass_yeskicks.png"width="100%" height="30%"></td>
        <td> <img src="Figures/lcp/mass_0_mass_1_error_4_Chirp_mass_nokicks.png"width="100%" height="30%"></td>
    </tr>
    <tr>
        <td>All systems</td>
        <td>Nokick</td>
    </tr>
    <tr>
        <td><img src="Figures/lcp/mass_0_mass_1_all_right_Chirp_Mass_yeskicks.png"></td>
        <td> <img src="Figures/lcp/mass_0_mass_1_all_right_Chirp_Mass_nokicks.png" ></td>
    </tr>
    <tr>
        <td>All systems</td>
        <td>Nokick</td>
    </tr>
</table>
</center><br>


# Final considerations and conclusions 
The model we built is able to predict with a measuread accuracy the right label for $q$ and $\mathcal{M}$.
We discovered that using a more sophisticated target for the training dataset, namely the $\mathcal{M}$, allowed the algorithm to yield far more accurate results as can be seen from the $\Delta q$ distributions and the KL and JS distances.

Furthermore, we showed how the model is subject to the random contributions of the supernova kicks in the binary stellar evolution: the model trained with the subset of systems not undergoing supernova explosions is indeed even more accurate.

As expected, the more peacked the output distribution is, the more accurate the prediction will be as we have extensively shown in graphs above.
This is a remarkable insight in a later usage of this algorithm: the predicion is almost surely certain whenever it yields a strong peak in its probability distribution.
Additionally we have seen that the more accurate probability distributions originate from metallicities greater, on average, than Z=0.006.

We also managed to rank the contributions of the input parameters. As expected the two more important features are the masses followed by the metallicity. The reason why we expect these quantities is that they are the main drivers of the underlying physical phenomena of the single stellar evolution.
Surprisingly the $\alpha$ parameter is far less important, for this algorithm, than all the other features. This is probably due to the fact that the $\alpha$ parameter is taken from a restricted range of values. Investigating a dataset with a broader spectrum of possible $\alpha$ could potentially increase its importance.

# Future developments
Possible ways to improve the current model consist in considering a larger dataset with a broader range of $\alpha$ values and a more evenly distributed input parameters space. Furthermore the binary evolution processes, such as Common Envelope and Roche Lobe Overflow, could be studied _a posteriori_ as we did for the random kick, in order to gain more insights on the physical reasons driving the algorithm in the guessing of the labels.
One additional attempt would use other more task oriented neural networks architectures such as Variational Autoencoders in order to build the target distributions.